In [1]:
from google.colab import drive
drive.mount('/content/drive')


import os
import time
import pprint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical

from PIL import Image
from osgeo import gdal

Mounted at /content/drive


In [ ]:
bc_data = r'/content/drive/MyDrive/Land Cover Classification with Sentinel-2/BC_input_data.CSV'
df_bc = pd.read_csv(bc_data)
on_data = r'/content/drive/MyDrive/Land Cover Classification with Sentinel-2/BC_input_data.CSV'
df_on = pd.read_csv(on_data)


In [ ]:
#load dataset in train/test variables
def load_dataset(df_train, df_test, num_classes):
    
    df_train = np.array(df_train).reshape( len(df_train), 1, -1 )
    trainX, trainy = df_train[:,:,0:-1], to_categorical(df_train[:,:,-1], num_classes=num_classes)  # one hot encode y
    
    df_test = np.array(df_test).reshape( len(df_test), 1, -1 )
    testX, testy = df_test[:,:,0:-1], to_categorical(df_test[:,:,-1], num_classes=num_classes)  # one hot encode y
    
    return trainX, trainy, testX, testy

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 10000
    n_features, n_outputs = trainX.shape[2], trainy.shape[-1]
    model = models.Sequential()
    model.add(layers.Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(1, n_features),padding='same'))
    model.add(layers.BatchNormalization())
    #model.add(layers.Dropout({{uniform(0, 1)}}))
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv1D(filters=64, kernel_size=1, activation='relu',padding='same'))
    model.add(layers.BatchNormalization())
    #model.add(layers.Dropout({{uniform(0, 1)}}))
    model.add(layers.Dropout(0.8))
    model.add(layers.MaxPooling1D(pool_size=1))
    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(n_outputs, activation='softmax'))
    opt = tf.keras.optimizers.Adam(learning_rate=0.1)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy
       
    
# run an experiment
def run_experiment(repeats=1):
    # load data
    trainX, trainy, testX, testy = load_dataset(df_on, df_bc, num_classes=20)  
    
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
        
    # summarize results
    m, s = np.mean(scores), np.std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [ ]:
st = time.time()
run_experiment(repeats=2)
print( f'Total Elapsed time = {(time.time() - st)/60 :.1f} min')

Epoch 1/50
512/512 [==============================] - 44s 82ms/step - loss: 1.0992 - accuracy: 0.6079
Epoch 2/50
512/512 [==============================] - 44s 86ms/step - loss: 0.9817 - accuracy: 0.6402
Epoch 3/50
512/512 [==============================] - 42s 82ms/step - loss: 0.9728 - accuracy: 0.6427
Epoch 4/50
512/512 [==============================] - 49s 96ms/step - loss: 0.9686 - accuracy: 0.6444
Epoch 5/50
512/512 [==============================] - 42s 82ms/step - loss: 0.9683 - accuracy: 0.6445
Epoch 6/50
512/512 [==============================] - 44s 87ms/step - loss: 0.9703 - accuracy: 0.6439
Epoch 7/50
512/512 [==============================] - 43s 85ms/step - loss: 0.9694 - accuracy: 0.6440
Epoch 8/50
512/512 [==============================] - 43s 84ms/step - loss: 0.9691 - accuracy: 0.6443
Epoch 9/50
512/512 [==============================] - 43s 85ms/step - loss: 0.9705 - accuracy: 0.6439
Epoch 10/50
512/512 [==============================] - 43s 85ms/step - loss: 0.970

In [ ]:
def learning_curve(history, epoch):

  # training vs validation accuracy
  epoch_range = range(1, epoch+1)
  plt.plot(epoch_range, history.history['accuracy'])
  plt.plot(epoch_range, history.history['val_accuracy'])
  plt.title('Model Accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'val'], loc='upper left')
  plt.show()

  # training vs validation loss
  plt.plot(epoch_range, history.history['loss'])
  plt.plot(epoch_range, history.history['val_loss'])
  plt.title('Model Loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'val'], loc='upper left')
  plt.show()

In [4]:
data <- pd.dataFrame(feature = rep(5, 5),
                   year = seq(2011, 2015), 
                   target = c(1, 0, 1, 0, 0))


NameError: ignored